In [1]:
import pandas as pd

In [2]:
# Income data df, zipcodes for NYC boroughs, and rat sightings df
income_df = pd.read_csv("us_income_zipcode.csv")
zipcode_borough = pd.read_csv("New_York_State_ZIP_Codes-County_FIPS_Cross-Reference.csv")
df = pd.read_csv("Rat_Sightings_20240917.csv", low_memory=False)

In [3]:
### Dropping all margin of error columns
def drop_margin_cols():
    lst = income_df.columns
    remove_list = []

    for col in lst:
        if "Margin of Error" in col:
            remove_list.append(col)
    return remove_list

lst = drop_margin_cols()
income_df.drop(lst, axis=1, inplace=True)

#Dropping other useless columns
income_df.drop(["Geography", "Geographic Area Name"], axis=1, inplace=True)
income_df

,ZIP,Households,"Households Less Than $10,000","Households $10,000 to $14,999","Households $15,000 to $24,999","Households $25,000 to $34,999","Households $35,000 to $49,999","Households $50,000 to $74,999","Households $75,000 to $99,999","Households $100,000 to $149,999",...,"Nonfamily Households $35,000 to $49,999","Nonfamily Households $50,000 to $74,999","Nonfamily Households $75,000 to $99,999","Nonfamily Households $100,000 to $149,999","Nonfamily Households $150,000 to $199,999","Nonfamily Households $200,000 or More",Nonfamily Households Median Income (Dollars),Nonfamily Households Mean Income (Dollars),Nonfamily Households Nonfamily Income in the Past 12 Months,Year
0,601,5397.0,33.2,15.7,23.9,13.4,8.7,3.5,0.7,0.9,...,3.6,0.3,0.9,0.0,0.0,0.0,9386.0,13044.0,15.0,2021.0
1,602,12858.0,27.1,12.7,20.5,14.3,10.9,8.2,3.3,2.0,...,6.0,3.2,1.8,0.0,0.0,0.0,11242.0,16419.0,20.1,2021.0
2,603,19295.0,32.1,13.4,17.2,9.9,9.3,10.2,4.1,2.5,...,4.4,4.7,1.1,0.5,0.6,0.2,10639.0,16824.0,34.9,2021.0
3,606,1968.0,28.4,13.3,23.3,15.4,11.0,7.7,0.7,0.3,...,4.6,3.0,0.0,0.0,0.0,0.0,15849.0,16312.0,13.0,2021.0
4,610,8934.0,20.5,13.2,23.3,15.7,11.5,9.8,3.4,1.7,...,7.0,3.0,0.0,0.0,0.0,0.0,12832.0,16756.0,14.5,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364993,99923,62.0,0.0,6.5,43.5,0.0,33.9,16.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,18155.0,17603.0,0.0,2011.0
364994,99925,345.0,2.0,4.9,18.0,12.2,13.9,22.3,6.4,18.0,...,10.9,26.1,0.7,8.7,2.9,0.0,33750.0,45545.0,14.5,2011.0
364995,99926,577.0,2.9,6.9,15.4,6.1,28.8,17.0,10.4,8.0,...,22.4,16.0,0.0,0.0,3.2,0.0,22083.0,34028.0,8.3,2011.0
364996,99927,43.0,20.9,27.9,20.9,23.3,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,12813.0,NaN,42.9,2011.0


In [4]:
#Dropping columns we don't need from borough df
zipcode_borough.drop(["State FIPS", "County Code", "File Date", "County FIPS"], axis=1, inplace=True)
zipcode_borough

,County Name,ZIP Code
0,Albany,12007
1,Albany,12009
2,Albany,12023
3,Albany,12041
4,Albany,12045
...,...,...
2538,Yates,14561
2539,Yates,14837
2540,Yates,14842
2541,Yates,14857


In [5]:
### COPY PASTED FROM CHECKPOINT 2 ###

# Keep only the necessary columns
df = df[['Created Date', 'Location Type', 'Incident Zip', 'Community Board', 'Borough', 'Latitude', 'Longitude']]

# Drop rows with unspecified boroughs
df = df[df['Borough'] != 'Unspecified']
df.reset_index(drop=True, inplace=True)

df.dropna(subset=['Created Date', 'Location Type', 'Incident Zip', 'Community Board', 'Borough', 'Latitude', 'Longitude'], inplace=True)

# Get unique values in 'Location Type'
unique_location_types = df['Location Type'].unique()

# Define mapping for residential categories
residential_mapping = {
    'Single Room Occupancy (SRO)': 'Single',
    '1-2 Family Dwelling': '1-2 Family Apartment',
    '1-2 FamilyDwelling': '1-2 Family Apartment',
    '3+ Family Apt. Building': '3+ Family',
    '3+ Family Apartment Building': '3+ Family',
    '3+ Family Apt.': '3+ Family',
    '3+Family Apt.': '3+ Family',
    '3+ Family Apt': '3+ Family',
    '1-3 Family Dwelling': '1-3 Family Dwelling',
    '1-3 Family Mixed Use Building': '1-3 Family Mixed',
    '1-2 Family Mixed Use Building': '1-2 Family Mixed',
    '3+ Family Mixed Use Building': '3+ Family Mixed',
    'Private House': 'Private House',
    'Apartment': 'Apartment',
    'Residential Property': 'Residential Property'
}

# Apply mapping to the 'Location Type' column for residential data
df['Location Type'] = df['Location Type'].replace(residential_mapping)
df['Incident Zip'] = df['Incident Zip'].astype(int)

df

,Created Date,Location Type,Incident Zip,Community Board,Borough,Latitude,Longitude
0,5/31/2024 0:10,Vacant Lot,11377,02 QUEENS,QUEENS,40.740375,-73.896454
1,6/1/2024 15:01,1-2 Family Apartment,11357,07 QUEENS,QUEENS,40.781667,-73.807381
2,6/6/2024 10:08,3+ Family,11385,05 QUEENS,QUEENS,40.705669,-73.908352
3,6/25/2024 11:34,3+ Family,10467,07 BRONX,BRONX,40.884066,-73.879225
4,6/27/2024 10:48,1-2 Family Apartment,11220,07 BROOKLYN,BROOKLYN,40.642061,-74.015862
...,...,...,...,...,...,...,...
253247,5/15/2024 18:38,Vacant Lot,10011,04 MANHATTAN,MANHATTAN,40.740951,-74.003244
253248,5/15/2024 6:58,3+ Family,10029,11 MANHATTAN,MANHATTAN,40.796473,-73.936536
253249,5/15/2024 19:29,1-2 Family Apartment,11209,10 BROOKLYN,BROOKLYN,40.636079,-74.034564
253250,5/15/2024 10:25,3+ Family Mixed,10031,09 MANHATTAN,MANHATTAN,40.824874,-73.948819


In [6]:
#Merging income_df and zipcode_borough
merged_df = income_df.merge(zipcode_borough, left_on="ZIP", right_on="ZIP Code")

#Filtering to only the 5 boroughs of NYC
merged_df = merged_df[(merged_df["County Name"] == "Bronx") | (merged_df["County Name"] == "Queens") | (merged_df["County Name"] == "New York") | (merged_df["County Name"] == "Kings") | (merged_df["County Name"] == "Richmond")]

#Dropping ZIP Code cause we don't need it (redundant column)
merged_df.drop("ZIP Code", axis=1, inplace=True)

#Fixing year to make it an int
merged_df["Year"] = merged_df["Year"].astype(int)

#Fixing households to make them ints
merged_df["Households"] = merged_df["Households"].astype(int)

#No more NA values!!!
merged_df.dropna()
merged_df

,ZIP,Households,"Households Less Than $10,000","Households $10,000 to $14,999","Households $15,000 to $24,999","Households $25,000 to $34,999","Households $35,000 to $49,999","Households $50,000 to $74,999","Households $75,000 to $99,999","Households $100,000 to $149,999",...,"Nonfamily Households $50,000 to $74,999","Nonfamily Households $75,000 to $99,999","Nonfamily Households $100,000 to $149,999","Nonfamily Households $150,000 to $199,999","Nonfamily Households $200,000 or More",Nonfamily Households Median Income (Dollars),Nonfamily Households Mean Income (Dollars),Nonfamily Households Nonfamily Income in the Past 12 Months,Year,County Name
1,10001,14273,7.2,3.4,5.4,3.3,7.4,11.5,11.2,14.9,...,12.2,13.0,15.7,11.4,19.0,89723.0,131592.0,20.1,2021,New York
2,10002,34737,13.0,11.8,12.9,10.0,9.1,11.2,9.2,9.2,...,10.0,9.4,10.1,4.3,8.0,32489.0,69003.0,34.8,2021,New York
3,10003,25181,6.8,2.2,3.3,4.3,4.2,11.9,6.9,13.3,...,14.7,7.6,14.2,10.7,27.7,105808.0,175535.0,28.5,2021,New York
4,10004,2113,2.2,0.0,0.9,1.6,0.4,2.0,6.1,13.5,...,3.1,10.6,19.9,21.0,38.7,159518.0,218597.0,25.7,2021,New York
5,10005,4782,0.9,0.0,0.1,1.4,2.1,9.2,6.1,15.2,...,11.7,5.0,17.2,18.4,41.4,185125.0,198121.0,21.7,2021,New York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22134,11692,5830,14.6,8.0,9.2,16.0,11.7,17.1,8.5,10.3,...,14.9,7.9,5.3,0.3,1.2,25617.0,39178.0,40.4,2011,Queens
22135,11693,4583,12.3,4.6,7.6,7.8,17.6,16.1,14.8,15.4,...,16.3,10.2,5.9,1.6,2.6,40742.0,57435.0,46.1,2011,Kings
22136,11693,4583,12.3,4.6,7.6,7.8,17.6,16.1,14.8,15.4,...,16.3,10.2,5.9,1.6,2.6,40742.0,57435.0,46.1,2011,Queens
22137,11694,8135,4.9,4.2,8.1,10.3,9.3,14.3,11.6,20.3,...,16.4,10.9,11.4,0.4,1.7,37194.0,52125.0,47.7,2011,Queens


In [7]:
#Merging invome with the rat sightings data
final_df = merged_df.merge(df, left_on="ZIP", right_on="Incident Zip")

# A function to keep only the important columns and mean/median income columns for final dataframe
def save_cols():
    columns = final_df.columns
    save_lst = []
    important = ["ZIP", "Households", "Year", "Created Date", "Location Type", "Community Board", "Borough", "Latitude", "Longitude"]

    for col in columns:
        if "Mean Income (Dollars)" in col or "Median Income (Dollars)" in col:
            save_lst.append(col)
        elif col in important:
            save_lst.append(col)
            
    return save_lst

lst = save_cols()
final_df = final_df[lst]

In [8]:
# Double check that we dont have NA values
final_df.dropna(inplace=True)
final_df.isna().sum()

ZIP                                                0
Households                                         0
Households Median Income (Dollars)                 0
Households Mean Income (Dollars)                   0
Families Median Income (Dollars)                   0
Families Mean Income (Dollars)                     0
Married-Couple Families Median Income (Dollars)    0
Married-Couple Families Mean Income (Dollars)      0
Nonfamily Households Median Income (Dollars)       0
Nonfamily Households Mean Income (Dollars)         0
Year                                               0
Created Date                                       0
Location Type                                      0
Community Board                                    0
Borough                                            0
Latitude                                           0
Longitude                                          0
dtype: int64

In [9]:
# Final income-rat sightings dataframe
final_df

,ZIP,Households,Households Median Income (Dollars),Households Mean Income (Dollars),Families Median Income (Dollars),Families Mean Income (Dollars),Married-Couple Families Median Income (Dollars),Married-Couple Families Mean Income (Dollars),Nonfamily Households Median Income (Dollars),Nonfamily Households Mean Income (Dollars),Year,Created Date,Location Type,Community Board,Borough,Latitude,Longitude
919,10002,34737,37093.0,76115.0,38815.0,82382.0,51689.0,101513.0,32489.0,69003.0,2021,5/16/2024 11:19,3+ Family,03 MANHATTAN,MANHATTAN,40.712894,-73.996101
920,10002,34737,37093.0,76115.0,38815.0,82382.0,51689.0,101513.0,32489.0,69003.0,2021,8/31/2024 20:15,Commercial Building,03 MANHATTAN,MANHATTAN,40.712995,-73.988529
921,10002,34737,37093.0,76115.0,38815.0,82382.0,51689.0,101513.0,32489.0,69003.0,2021,8/15/2024 5:24,3+ Family,03 MANHATTAN,MANHATTAN,40.712353,-73.993947
922,10002,34737,37093.0,76115.0,38815.0,82382.0,51689.0,101513.0,32489.0,69003.0,2021,6/4/2024 20:19,Office Building,03 MANHATTAN,MANHATTAN,40.722009,-73.993376
923,10002,34737,37093.0,76115.0,38815.0,82382.0,51689.0,101513.0,32489.0,69003.0,2021,5/8/2024 17:31,3+ Family,03 MANHATTAN,MANHATTAN,40.716111,-73.992998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2790230,11435,17950,54086.0,65131.0,57789.0,70488.0,67135.0,78335.0,39375.0,51434.0,2011,4/26/2024 17:54,1-2 Family Apartment,08 QUEENS,QUEENS,40.711187,-73.810153
2790231,11435,17950,54086.0,65131.0,57789.0,70488.0,67135.0,78335.0,39375.0,51434.0,2011,5/1/2024 13:05,1-2 Family Apartment,12 QUEENS,QUEENS,40.687113,-73.799311
2790232,11435,17950,54086.0,65131.0,57789.0,70488.0,67135.0,78335.0,39375.0,51434.0,2011,5/2/2024 20:37,Commercial Building,12 QUEENS,QUEENS,40.701386,-73.804111
2790233,11435,17950,54086.0,65131.0,57789.0,70488.0,67135.0,78335.0,39375.0,51434.0,2011,5/4/2024 18:17,1-2 Family Apartment,12 QUEENS,QUEENS,40.694572,-73.809710
